In [51]:
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

# Import prompts
from prompts import *

# Set up LLM to user
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)

In [52]:
# Chain to generate initial answer
baseline_response_prompt_template = PromptTemplate.from_template(BASELINE_PROMPT_WIKI)
baseline_response_chain = baseline_response_prompt_template | llm | StrOutputParser()

In [53]:
# Chain to generate a question template for verification answers
verification_question_template_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_TEMPLATE_PROMPT_WIKI)
verification_question_template_chain = verification_question_template_prompt_template | llm | StrOutputParser()

In [54]:
# Chain to generate the verification questions
verification_question_generation_prompt_template = PromptTemplate.from_template(VERIFICATION_QUESTION_PROMPT_WIKI)
verification_question_generation_chain = verification_question_generation_prompt_template | llm | StrOutputParser()

In [55]:
# Chain to execute the verification
execution_prompt_search_llm = PromptTemplate.from_template(EXECUTE_PLAN_PROMPT_SEARCH_TOOL)
execution_prompt_llm_chain = execution_prompt_search_llm | llm | StrOutputParser()

Answer the following question correctly based on the provided context. The question could be tricky as well, so think step by step and answer it correctly.

Context: {search_result}

Question: {verification_question}

Answer:


In [56]:
from langchain_community.tools import DuckDuckGoSearchResults

search = DuckDuckGoSearchResults()

In [57]:
verification_chain = RunnablePassthrough.assign(
    split_questions=lambda x: x['verification_questions'].split("\n"),
) | RunnablePassthrough.assign(
    answers = (lambda x: [
        {"search_result":search.run(q),"verification_question": q} for q in x['split_questions']])
| execution_prompt_llm_chain.map()
) \
| (lambda x: "\n".join(["Question: {} Answer: {}\n".format(question, answer) for question, answer in zip(x['split_questions'], x['answers'])]))# Create final refined response



In [58]:
# Chain to generate the final answer
final_answer_prompt_template = PromptTemplate.from_template(FINAL_REFINED_PROMPT)
final_answer_chain = final_answer_prompt_template | llm | StrOutputParser()




In [60]:
# Putting everything together, a final chain

chain = RunnablePassthrough.assign(
    baseline_response=baseline_response_chain
) \
| RunnablePassthrough.assign(
    verification_question_template=verification_question_template_chain
) \
| RunnablePassthrough.assign(
    verification_questions=verification_question_generation_chain
) \
| RunnablePassthrough.assign(
    verification_answers=verification_chain
) \
| RunnablePassthrough.assign(
    final_answer=final_answer_chain
)

response = chain.invoke({"original_question": "Who are some politicians born in city of Boston?"},config={'callbacks': [ConsoleCallbackHandler()]})
print(response)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "original_question": "Who are some politicians born in city of Boston?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<baseline_response>] Entering Chain run with input:
{
  "original_question": "Who are some politicians born in city of Boston?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<baseline_response> > chain:RunnableParallel<baseline_response>] Entering Chain run with input:
{
  "original_question": "Who are some politicians born in city of Boston?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<baseline_response> > chain:RunnableParallel<baseline_response> > chain:RunnableSequence] Entering Chain run with input:
{
  "original_question": "Who are some politicians born in city of Boston?"
}
[chain/start] [chain:RunnableSequence > chain:RunnableAssign<baseline_response> > chain:RunnableParallel<baseline_response> > chain:RunnableSequence > prompt:PromptTempl